En este proceso se explica la forma en que pueden descargarse los datos disponibles en la [API](https://bchapi-am.developer.azure-api.net/) del Banco Central de Honduras (BCH), utilizando Julia.

# Acceso a los Datos

Una vez que se crea el registro de usuario (usar el botón "Suscribirse" en el [sitio web](https://bchapi-am.developer.azure-api.net/), puede verse el procedimiento de registro e inicio de sesión en [YouTube](https://www.youtube.com/watch?v=8ZBllMSsKw4)), se necesita obtener una clave (ver explicación en [YouTube](https://www.youtube.com/watch?v=mV90s74OCfc)), ejecutando una consulta al catálogo de indicadores. Favor sustituir en el código la clave actual por su clave asignada.

Para conocer los datos disponibles, se debe ejecutar una consulta al catálogo de indicadores, que contiene información sobre las series disponibles:

* Nombre (código)
* Descripción
* Periodicidad
* Grupo
* Correlativo del Grupo


In [ ]:
#import Pkg; Pkg.add("IMFData")
using Azure,Chain,CSV,DataFrames,DataFramesMeta,Dates,HTTP,JSON,JSON3

wd = @__DIR__
clave = "ff34cff7b0024ea39eb565fccb9f03b6" #Favor ingresar la clave proporcionada.

# Formatos de consulta

## 1. Consulta de catálogo de indicadores

Consulta el catálogo de indicadores publicados mediante la Web-API, asimismo, se obtiene el detalle de la información del indicador: Id, nombre, descripción, grupo, correlativo de grupo y periodicidad. Ningún parámetro es obligatorio.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores[?formato][&nombre][&descripcion][&periodicidad][&ordenamiento][&grupo][&correlativogrupo][&omitir][&reciente]`

Este es el listado de todos los indicadores disponibles:


In [ ]:
function get_indicators(key)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/?formato=Json"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  df = df[!,[:Id,:Nombre,:Descripcion,:CorrelativoGrupo,:Grupo,:Periodicidad]]
end

function get_indicators_filter(key,url)
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  df = df[!,[:Id,:Nombre,:Descripcion,:CorrelativoGrupo,:Grupo,:Periodicidad]]
end

In [ ]:
key = clave
get_indicators(key)

Consultar solamente los indicadores filtrados por nombre (puede usarse partes del nombre), por ejemplo, "ESR":


In [ ]:
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/?formato=Json&nombre=ESR"
get_indicators_filter(key,url)

Esto aplica para el resto de los campos; un ejemplo de consulta con varios filtros:


In [ ]:
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/?formato=Json&nombre=PIBA&correlativogrupo=10&Descripcion=Constantes"
get_indicators_filter(key,url)

Nota: tomar en cuenta para todas las consultas por filtro que si la ruta no existe, el código genera un error.

## 2. Consulta cifras por Id de indicador

Consulta las cifras para un indicador en específico obtenido por el Id del indicador. El parámetro "Id" es un campo obligatorio el resto de criterios quedan a consideración del usuario. Si no se especifican parámetros, se obtendrán todos los datos disponibles para el indicador requerido, lo que podría demorar la obtención de la consulta, en ese sentido lo que se sugiere afinar la búsqueda a los criterios requeridos.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/{id}/cifras[?formato][&fechaInicio][&fechaFinal][&valorMinimo][&valorMaximo][&ordenamiento][&omitir][&reciente]`


In [ ]:
function get_data_indicator(key,id)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/" *  string(id) * "/cifras"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  myFormat = Dates.DateFormat("yyy-mm-ddTHH:MM:SS")
  df.Fecha = @. Dates.Date(Dates.DateTime.(df.Fecha ,myFormat))
  df = df[!,[:IndicadorId,:Nombre,:Descripcion,:Fecha,:Valor]]
  DataFrames.sort!(df,[:Fecha],rev=true)
  return df
end

function get_data_indicator_filter(key,url,id)
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  myFormat = Dates.DateFormat("yyy-mm-ddTHH:MM:SS")
  df.Fecha = @. Dates.Date(Dates.DateTime.(df.Fecha ,myFormat))
  df = df[!,[:IndicadorId,:Nombre,:Descripcion,:Fecha,:Valor]]
  DataFrames.sort!(df,[:Fecha],rev=true)
  return df
end

function get_data_indicators(key,ids)
  ids = collect(ids)
  df = DataFrames.DataFrame()
  for i in ids
    df = vcat(df,get_data_indicator(key,i))
  end
  return df
end

Consultar solo un indicador:


In [ ]:
id = 1500
get_data_indicator(key,id)

Consultar varios indicadores, usando rangos:


In [ ]:
ids = 1:10
get_data_indicators(key,ids)

Consultar varios indicadores, usando IDs no contínuas:


In [ ]:
ids = 1,7,25,36
get_data_indicators(key,ids)

Consultar solo un indicador, usando filtros:


In [ ]:
id = 36
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/" * string(id) * "/cifras?FechaInicio=2023-03-01&FechaFinal=2023-09-01"
get_data_indicator_filter(key,url,id)

## 3. Consulta Indicadores por grupo

Consulta un conjunto de indicadores publicados mediante la Web-API por grupo de indicadores. El parámetro "grupo" es un campo obligatorio.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/{grupo}[?formato]`


In [ ]:
function get_groups(key)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores?formato=Json"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  DataFrames.sort!(df,:CorrelativoGrupo)
  df = df[!,[:CorrelativoGrupo,:Grupo,:Id,:Nombre,:Descripcion,:Periodicidad]]
  return df
end

function get_groups_filter(key,url)
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  DataFrames.sort!(df,:Id)
  return df
end

Consultar lista de todos los indicadores:


In [ ]:
get_groups(key)

Consultar componentes de grupo por filtro (en este caso debe especificarse el nombre completo del grupo, de acuerdo con la tabla anterior; caso contrario, se generan errores):


In [ ]:
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/ESR-PIBT-PROD-OG-CONST?formato=Json"
get_groups_filter(key,url)

## 4. Consulta cifras por grupo de indicadores

Consultar las cifras de todos los indicadores por grupo de indicadores. El parámetro "grupo" es un campo obligatorio.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/{grupo}/cifras[?formato]`

Debe especificarse el nombre completo del (los) grupo(s), de acuerdo con la tabla anterior; caso contrario, se generan errores.


In [ ]:
function get_data_group(key,group)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" *  group * "/cifras"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  myFormat = Dates.DateFormat("yyy-mm-ddTHH:MM:SS")
  df.Fecha = @. Dates.Date(Dates.DateTime.(df.Fecha ,myFormat))
  df[!,:Grupo] .= group
  df = df[!,[:Grupo,:IndicadorId,:Nombre,:Descripcion,:Fecha,:Valor]]
  DataFrames.sort!(df,[:IndicadorId,:Fecha],rev=[false,true])
  return df
end

function get_data_groups(key,groups)
  df = DataFrames.DataFrame()
  for i in 1:size(groups)[1]
    df = vcat(df,get_data_group(key,groups[i]))
  end
  return df
end

Consultar cifras de un solo grupo:


In [ ]:
group = "ESR-PIBT-PROD-OG-CONST"
get_data_group(key,group)

Consultar cifras de varios grupos:


In [ ]:
groups = ["ESR-PIBT-PROD-OG-CONST","ESE-PII-01"]
get_data_groups(key,groups)

## 5. Consulta por fecha de registro

Consulta el valor de un indicador para una fecha específica. Todos los parámetros son obligatorios.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/{id}/cifras/{fecha}[?formato]`


In [ ]:
function get_data_date(key,id,date)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/" * string(id) * "/cifras/" * date * "?formato=JSON"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  myFormat = Dates.DateFormat("yyy-mm-ddTHH:MM:SS")
  df.Fecha = @. Dates.Date(Dates.DateTime.(df.Fecha ,myFormat))
  df[!,:Grupo] .= group
  df = df[!,[:Grupo,:IndicadorId,:Nombre,:Descripcion,:Fecha,:Valor]]
  DataFrames.sort!(df,[:IndicadorId,:Fecha],rev=[false,true])
  return df
end

Consultar solamente un Id y una fecha:


In [ ]:
id = 1
date = "2023-03-01"
get_data_date(key,id,date)

## 6. Consulta de información por Id de indicador

Consulta el detalle de la información del indicador: Id, nombre, descripción, grupo, correlativo de grupo y periodicidad. El parámetro "Id" es un campo obligatorio.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/{id}/cifras/{fecha}[?formato]`


In [ ]:
function get_info_id(key,id)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/" * string(id) * "?formato=JSON"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  df = df[!,[:Id,:Nombre,:Descripcion,:Periodicidad,:CorrelativoGrupo,:Grupo]]
  return df
end

function get_info_ids(key,ids)
  ids = collect(ids)
  df = DataFrames.DataFrame()
  for i in ids
    df = vcat(df,get_info_id(key,i))
  end
  return df
end

Consultar solamente un Id:


In [ ]:
id = 1
get_info_id(key,id)

Consultar varios Ids, rango:


In [ ]:
ids = 1:3
get_info_ids(key,ids)

Consultar varios Ids, números:


In [ ]:
ids = 1,7,25
get_info_ids(key,ids)

## 7. Conteo de cifras para cada indicador

Consulta los indicadores publicados mediante la Web-API devolviendo el conteo de registros disponibles para cada uno. Todos los filtros son opcionales.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/conteos[?formato][&nombre][&descripcion][&periodicidad][&ordenamiento][&conteoMinimo][&conteoMaximo][&ordenamientoConteo][&grupo][&correlativogrupo][&omitir][&reciente]`

Esta forma de consulta permite filtrar por valores incompletos de los elementos nombre, descripcion y grupo.


In [ ]:
function get_count_gids(key,url)
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  df = df[!,[:Id,:Nombre,:Descripcion,:Periodicidad,:CorrelativoGrupo,:Grupo,:Conteo]]
  return df
end

Filtro por nombre del indicador:


In [ ]:
filtro = "EC-"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/conteos?formato=Json&nombre=" * filtro
get_count_gids(key,url)

Filtro por descripción del indicador:


In [ ]:
filtro = "cambio"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/conteos?formato=Json&descripcion=" * filtro
get_count_gids(key,url)

Filtro por periodicidad del indicador:

- Diario;
- Mensual;
- Trimestral; y
- Anual.


In [ ]:
filtro = "Mensual"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/conteos?formato=Json&periodicidad=" * filtro
get_count_gids(key,url)

Filtro por grupo del indicador:


In [ ]:
filtro = "ODA"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/conteos?formato=Json&grupo=" * filtro
get_count_gids(key,url)

Filtro por correlativo de grupo del indicador:


In [ ]:
filtro = 10
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/conteos?formato=Json&correlativogrupo=" * string(filtro)
get_count_gids(key,url)

## 8. Conteo de cifras para un indicador

Consulta el conteo de registros disponibles para un indicador en específico. El parámetro "Id" es un campo obligatorio.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/{id}/conteos[?formato]`


In [ ]:
function get_count_id1(key,id)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/" * string(id) * "/conteos?formato=Json"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  df = df[!,[:Id,:Nombre,:Descripcion,:Periodicidad,:CorrelativoGrupo,:Grupo,:Conteo]]
  return df
end

In [ ]:
id = 110
get_count_id1(key,id)

# Grupos

Las variables están categorizadas en 8 grupos principales y 54 subgrupos (ESE-BP-01 y ESE-CEB-01 contienen variables de periodicidad anual o trimestral, y EM-TPM-01 tiene variables de periodicidad diaria o mensual). La nomenclatura utilizada permite dividir estos en un máximo de cinco niveles, tomando en cuenta el separador (-). Se explicarán los mismos a continuacion, ordenando de acuerdo al número de niveles (número de subgrupos).


In [ ]:
df = get_groups(key)
df_groups = @chain df begin
  DataFrames.groupby(
    # [:Grupo,:CorrelativoGrupo,:Periodicidad])
    # [:Grupo])
    [:Grupo,:Periodicidad])
  DataFrames.combine(
    nrow => :Conteo)
end
DataFrames.sort!(df_groups,:Conteo,rev=true)
df_groups = hcat(
  df_groups,
  DataFrames.select(
    df_groups, 
    :Grupo =>
      ByRow(x -> get.(Ref(split(x, '-')), 1:5, missing)) =>
      [:Gr1,:Gr2,:Gr3,:Gr4,:Gr5]))
CSV.write(
    wd .* "/api/grupos_julia.csv",
    delim = ";",
  df_groups);

Todos los grupos pueden clasificarse en al menos dos subgrupos:


In [ ]:
df_groups
DataFrames.describe(
  df_groups, 
  :nmissing, 
  length => :length,
  cols = [:Gr1,:Gr2,:Gr3,:Gr4,:Gr5])

In [ ]:
function count_groups(x)
  c = 5
  @inbounds for i in eachindex(x)
      if ismissing(x[i])
          c -= 1
      end
  end
  return c
end

In [ ]:
DataFrames.transform!(
  df_groups, 
  AsTable(Between(:Gr1,:Gr5)) => ByRow(count_groups) => :N_Subgrupos)
DataFrames.sort!(df_groups,[:Grupo,:N_Subgrupos],rev=[true,true])

Los 8 grupos principales mencionados permiten consolidar los archivos de acuerdo con el origen de los datos en la página web del BCH, que en su mayoría pueden consultarse en los reportes dinámicos. A su vez, estos grupos pueden subdividirse en 25 subgrupos:

1. EOM = Estadísticas de Operaciones Monetarias

1.1. OMA = Operaciones de Mercado Abierto

Cuatro niveles de agregación:

- 01 = Tasas de Rendimiento Diario de Valores Gubernamentales
- 02 = Créditos (Niv_Descr_4 = 1-3) e Inversiones (Niv_Descr_4 = 4-6): Monto, Número de Solicitudes y Tasa
- 03 = Monto, Número de Operaciones y Tasa
- 04 = Monto, Número de Operaciones y Tasa (Dólares - Lempiras)

2. ESR = Estadísticas del Sector Real

2.1. COU = Cuadro de Oferta y Utilización

2.2. CCI = Clasificación Cruzada Industria

2.3. VAB = Valor Agregado Bruto

2.4. PIBA = PIB Anual

2.5. PIBT = PIB Trimestral

2.6. ODA = Oferta y Demanda Agregada

2.7. IMAE = Índice Mensual de la Actividad Económica

3. ESE = Estadísticas del Sector Externo
4. EMF = Estadísiticas Monetarias y Financieras (Sector Fiscal)
5. EM = Estadísticas Monetarias (TPM y RIN)
6. ESP = Estadísticas de Sistema de Pagos
7. EC = Estadísticas de Tipo de Cambio
8. EP = Estadísticas de Precios

Los grupos se subdividen en 25 subgrupos:
